In [1]:
import rdflib

#rdflib library is imported

### Getting the triples from the graph


In [2]:
g = rdflib.Graph()
g.parse(r"C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/statements.ttl", format="turtle")
query = """SELECT ?s ?p ?o WHERE
{
   ?s ?p ?o.
     
   
}   
 """
results = g.query(query)


### The function get_delimiter helps in splitting the subject in each triple by the last '/' from the right


In [3]:
def get_delimiter(url):
    # if '#' in url:
    #     delimiter = '#'
    # else:
    delimiter = '/'
    return delimiter

 ### First we create a dataframe in order to process the triples, having the columns 'URL','URL_FIRST_PART','URL_LAST_PART','PREDICATE', 'OBJECT','WORDS','ENTITIES','REPLACED_WORD'. 'URL' is the URL of the subject ,'URL_FIRST_PART' the part of the subject before the last '/' from the right ,'URL_LAST_PART' the part of the subject after the last '/' from the right,'PREDICATE' the predicate of the triples, 'OBJECT' the object of the triples. 'WORDS' contains the 'URL_LAST_PART' without special characters.  'ENTITIES' contains NER tagging for the data in 'WORDS' using SpaCy library.'REPLACED_WORD' contains the correct replacement in place of 'URL_LAST_PART', which will then be appended to 'URL_FIRST_PART' to have the correct subject URL.

In [5]:
import pandas as pd
df = pd.DataFrame(columns = ['URL','URL_FIRST_PART','URL_LAST_PART','PREDICATE', 'OBJECT','WORDS','ENTITIES','REPLACED_WORD'])
l = []
for result in results:
    subject = str(result).split(',')[0]
    predicate = str(result).split(',')[1]
    obj = str(result).split(',')[2]
    
    
    
    uri = rdflib.term.URIRef(subject)
    uri_str = str(uri)
    temp = uri_str.split("'")[1]
    delimiter = get_delimiter(uri_str)
    shortened_uri_str = (uri_str.split(delimiter)[-1]).split("'")[0]
    shortened_uri_str_2 = '/'.join(uri_str.split(delimiter)[:-1]).split("'")[1]
    
        
    temp_str2 = str(predicate).split('(')[0]
    uri = rdflib.term.URIRef(predicate)
    uri_str = str(uri)
    temp2 = uri_str.split("'")[1]
   
    uri = rdflib.term.URIRef(obj)
    uri_str = str(uri)
    temp3 = uri_str.split("'")[1]
    
    
    
   
   
    df = df.append({'URL': temp,'URL_FIRST_PART' : shortened_uri_str_2, 'URL_LAST_PART':  shortened_uri_str , 'PREDICATE':temp2, 'OBJECT':temp3}, ignore_index = True)
    
        


 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') does not look like a valid URI, trying to serialize this will break.
 rdflib.term.URIRef('http://fraunhofer.de/MarketActivities/Researcher')) does not look like a valid URI, trying to serialize this will break.
C:\Users\mahar\AppData\Local\Temp\ipykernel_22344\4047445619.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'URL': temp,'URL_FIRST_PART' : shortened_uri_str_2, 'URL_LAST_PART':  shortened_uri_str , 'PREDICATE':temp2, 'OBJECT':temp3}, ignore_index = True)
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') does not look like a valid URI, trying to serialize this will break.
 rdflib.term.URIRef('http://www.semanticweb.org/neunsinger/ontologies/2022/2.0/HydrogenOntology#Bike')) does not look like a valid URI, trying to serialize this will break.
C:\Users\mahar\AppData\Local\Temp\ipyke

### We filter out the rows having subject URLs containing the term 'hydrogen' in them.

In [6]:
First_filtered_df = (df[df['URL_LAST_PART'].str.contains('hydrogen',case = False)])
First_filtered_df.reset_index(drop = True, inplace = True)
First_filtered_df.to_csv('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.csv')




### We find out all the special characters in the last part of all the subject URLs.

In [ ]:
import regex as re
list5 = []
set5 = set()
for _,row in First_filtered_df.iterrows():
    if len(re.findall('[^A-Za-z0-9_]',row['URL_LAST_PART']))!=0:
        list5.append(re.findall('[^A-Za-z0-9_]',row['URL_LAST_PART']))
for internal_list in list5:
    for item in internal_list:
        set5.add(item)
unique_list = list(set5)
print(unique_list)

### If the special characters are '.' or '-' in the last part of the subject URL, we simply replace them by '_'.

In [8]:
for _,row in First_filtered_df.iterrows():
    for char in row['URL_LAST_PART']:
        if char == '.' or char == '-':
            row['URL_LAST_PART'] = row['URL_LAST_PART'].replace(char,'_')

        

### If they are special characters like &,$,',' etc., we find them in formats such as '%24','%3F' etc in our Turtle files. We find all the special characters present in the format %[A-Z0-9]{2} in all the rows of the file, and replace them with '_'. 

In [ ]:
import regex as re
list5 = []
set5 = set()
for _,row in First_filtered_df.iterrows():
    if len(re.findall('%[A-Z0-9]{2}',row['URL_LAST_PART']))!=0:
        #print(_,row['URL_LAST_PART'])
        list5.append(re.findall('%[A-Z0-9]{2}',row['URL_LAST_PART']))
for internal_list in list5:
    for item in internal_list:
        set5.add(item)
unique_list = list(set5)
#print(unique_list)
for char in unique_list:
    for _,row in First_filtered_df.iterrows():
        if char in row['URL_LAST_PART']:
            row['URL_LAST_PART'] = row['URL_LAST_PART'].replace(char,'_')
print(First_filtered_df['URL_LAST_PART'])



### We then join the words in last parts of the subject URLs which are currently separated by '_', and then rejoin them with a space between them in the column 'WORDS'

In [10]:
for _,row in First_filtered_df.iterrows():
    row['URL_LAST_PART'].strip()
    row['WORDS'] = row['URL_LAST_PART'].split('_')
    row['WORDS'] = ' '.join(row['WORDS'])


### We then use the spaCy library and the 'en_core_web_sm' English language model on the ROWS column. 'en_core_web_sm' includes tokenization, part-of-speech tagging, named entity recognition, and other linguistic features.

In [11]:
import spacy

nlp = spacy.load('en_core_web_sm')

def perform_ner(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))
    return entities
for _,row in First_filtered_df.iterrows():
    text = str(row['WORDS'])
    row['ENTITIES'] = perform_ner(text)
    
    

### We then fill the REPLACED_WORD column based on conditions fulfilled by the words in the WORDS column for each row.

In [ ]:
import pandas as pd

# Assuming you have a DataFrame named 'First_filtered_df' with columns 'WORDS' and 'REPLACED_WORD'

# Filter the rows that contain the word 'train' (case insensitive)
vehicle_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('vehicle', case=False)]
Technology_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('Technology', case=False)]
Infrastructure_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('Infrastructure', case=False)]
Transportation_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('transport', case=False)]
train_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('train', case=False)]
tank_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('tank', case=False)]
bus_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('bus', case=False)]
plane_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('plane', case=False)]
fuel_cell_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('fuel', case=False) & First_filtered_df['WORDS'].str.contains('cell', case=False)]
fuel_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('fuel', case=False) & ~First_filtered_df['WORDS'].str.contains('cell', case=False)]
car_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('car', case=False)& ~First_filtered_df['WORDS'].str.contains('carbon', case=False) ]
fleets_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('fleet', case=False)]
NEXO_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('NEXO', case=False) ]
Hyets_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('HyET Hydrogen', case=False)]
Alliance_rows = First_filtered_df[First_filtered_df['WORDS'].str.contains('Clean', case=False)&First_filtered_df['WORDS'].str.contains('Hydrogen', case=False) &First_filtered_df['WORDS'].str.contains('Alliance', case=False)]

 



# Assign 'Train' to the 'REPLACED_WORD' column in the filtered rows
First_filtered_df.loc[train_rows.index, 'REPLACED_WORD'] = 'Train'
First_filtered_df.loc[tank_rows.index, 'REPLACED_WORD'] = 'Tank'
First_filtered_df.loc[bus_rows.index, 'REPLACED_WORD'] = 'Bus'
First_filtered_df.loc[plane_rows.index, 'REPLACED_WORD'] = 'Plane'
First_filtered_df.loc[fuel_cell_rows.index, 'REPLACED_WORD'] = 'Fuel_Cell'
First_filtered_df.loc[car_rows.index, 'REPLACED_WORD'] = 'Car'
First_filtered_df.loc[vehicle_rows.index, 'REPLACED_WORD'] = 'Vehicle'
First_filtered_df.loc[NEXO_rows.index, 'REPLACED_WORD'] = 'NEXO'
First_filtered_df.loc[fuel_rows.index, 'REPLACED_WORD'] = 'Fuel_Cell'
First_filtered_df.loc[fleets_rows.index, 'REPLACED_WORD'] = 'Fleet'
First_filtered_df.loc[Hyets_rows.index, 'REPLACED_WORD'] = 'HyET_Hydrogen'
First_filtered_df.loc[Alliance_rows.index, 'REPLACED_WORD'] = 'European_Clean_Hydrogen_Alliance'
First_filtered_df.loc[Technology_rows.index, 'REPLACED_WORD'] = 'Technology'
First_filtered_df.loc[Infrastructure_rows.index, 'REPLACED_WORD'] = 'Infrastructure'
First_filtered_df.loc[Transportation_rows.index, 'REPLACED_WORD'] = 'Transportation'


# Print the filtered DataFrame



## We export the results to First_filtered_df.csv

In [33]:
First_filtered_df.to_csv('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.csv')

### If there are any empty entries under REPLACED_WORDS column in the csv file, we fill it with 'Hydrogen'.

In [34]:
FFD = pd.read_csv('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.csv')
NAN_rows = FFD[FFD['REPLACED_WORD'].isna()]
FFD.loc[NAN_rows.index, 'REPLACED_WORD'] = 'Hydrogen'
FFD.to_csv('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.csv')

### We first import the data from First_filtered_df.xlsx, which has been created from First_filtered_df.csv. Then we append the subject URL first part, the '/', and the REPLACED_WORD in place of the URL_LAST_PART, and export it to statements1.txt

In [ ]:
FFD = pd.read_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')

# Create a new column 'MODIFIED_URL' based on the values of other columns
FFD['MODIFIED_URL'] = FFD.apply(lambda row: row['URL_FIRST_PART'] + '/' + row['REPLACED_WORD'], axis=1)
NH = df[~df['URL_LAST_PART'].str.contains('hydrogen',case = False)]
NH['MODIFIED_URL'] = NH.apply(lambda row: row['URL_FIRST_PART'] + '/' + row['URL_LAST_PART'], axis=1)

# Display the modified DataFrame
print(FFD)
# FFD.to_csv('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.csv')
with open('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/statements1.txt','w') as page:
    for _,row in FFD.iterrows():
        page.writelines('<'+row['MODIFIED_URL']+'>' + ' '+ '<'+row['PREDICATE']+'>'+ ' '+ '<'+row['OBJECT']+'>.\n')
    for _,row in NH.iterrows():
        page.writelines('<'+row['MODIFIED_URL']+'>' + ' '+ '<'+row['PREDICATE']+'>'+ ' '+ '<'+row['OBJECT']+'>.\n')
    
    



### Similarly, we search for 'Green Hydrogen','Peroxide','Hydrogen Projects', 'Hydrogen Production' in the WORDS column for each row and replace them accordingly.

In [18]:

FFD = pd.read_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')
green_rows = FFD[(FFD['REPLACED_WORD']== 'Hydrogen') & (FFD['WORDS'].str.contains('Green',case = False)|FFD['WORDS'].str.contains('carbon free',case = False)|FFD['WORDS'].str.contains('zero carbon',case = False)|FFD['WORDS'].str.contains('low carbon',case = False)|FFD['WORDS'].str.contains('renewable',case = False))]
green_rows.count()
FFD.loc[green_rows.index,'REPLACED_WORD'] = 'Green_Hydrogen'




In [20]:
FFD.to_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')

In [22]:
Peroxide_rows = FFD[(FFD['WORDS'].str.contains('Peroxide',case = False))]
Peroxide_rows
FFD.loc[Peroxide_rows.index, 'REPLACED_WORD'] = 'Hydrogen_Peroxide'
FFD.to_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')

In [23]:
Production_rows = FFD[(FFD['WORDS'].str.contains('production',case = False))& (FFD['REPLACED_WORD'] == 'Hydrogen')]
FFD.loc[Production_rows.index, 'REPLACED_WORD'] = 'Hydrogen_Production'
FFD.to_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')

In [24]:
Production_rows = FFD[(FFD['WORDS'].str.contains('project',case = False))& (FFD['REPLACED_WORD'] == 'Hydrogen')]
FFD.loc[Production_rows.index, 'REPLACED_WORD'] = 'Hydrogen_Projects'
FFD.to_excel('C:/Users/mahar/OneDrive/Desktop/Fraunhofer/Hydrogen/First_filtered_df.xlsx')